<a href="https://colab.research.google.com/github/ukab23/-Simple-Linear-Regression/blob/master/cdqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
import io
data = pd.read_csv('content1.csv')
data.head()

,Unnamed: 0,title,publication,author,content,paragraphs
0,0,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,washington congressional republican new fear c...,['washington congressional republican new fear...
1,1,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,bullet shell get counted blood dry votive cand...,['bullet shell get counted blood dry votive ca...
2,2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,walt disney’s bambi opened 1942 critic praised...,['walt disney’s bambi opened 1942 critic prais...
3,3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,death may great equalizer isn’t necessarily ev...,['death may great equalizer isn’t necessarily ...
4,4,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,seoul south korea north korea’s leader kim sai...,['seoul south korea north korea’s leader kim s...


In [0]:
from cdqa.utils.converters import df2squad
# Converting dataframe to SQuAD format
json_data = df2squad(df=data, squad_version='v1.1', output_dir='.', filename='data')

2105it [00:27, 130.94it/s]

In [0]:
import os
from ast import literal_eval
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline.cdqa_sklearn import QAPipeline

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
from cdqa.utils.download import download_model
download_model(model='bert-squad_1.1', dir='./models')


bert_qa_vCPU-sklearn.joblib already downloaded


In [0]:
def text_list(text):
    text = text.split('.')
    return text

In [0]:
punctuations = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~—”“'
stp_wrd =  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're",
           "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he',
           'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
           "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
           'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
           'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
           'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
           'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with',
           'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
           'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
           'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
           'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
           'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
           'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've",
           'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn',
           "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
           "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't",
           'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
           'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [0]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def process_text(text):
    nopunc = [char for char in text if char not in punctuations]
    nopunc = ''.join(nopunc)
    clean_words = [word.lower() for word in nopunc.split() if word.lower() not in stp_wrd]
    clean_words = [lemmatizer.lemmatize(lem) for lem in clean_words]
    clean_words = " ".join(clean_words)
    return clean_words

In [0]:
data['content'] = data['content'].apply(process_text)

In [0]:
data['paragraphs'] = data['content'].apply(text_list)

In [0]:
data.to_csv('content.csv')

In [0]:
df = pd.read_csv('content.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df = df[['title','paragraphs']]
df.head()

,title,paragraphs
0,House Republicans Fret About Winning Their Hea...,[eager to avoid an ugly political pileup repub...
1,Rift Between Officers and Residents as Killing...,[gola white who wa beside her daughter when sh...
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",[the question came like gunfire in which direc...
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",[he had hinted that his time wa short in the l...
4,Kim Jong-un Says North Korea Is Preparing to T...,[after the north’s satellite launch in februar...


In [0]:
df['paragraphs'][4]

['after the north’s satellite launch in february south korean defense official said the unha rocket used in the launch if successfully reconfigured a a missile could fly more than 7 400 mile with a warhead of 1 100 to 1 300 pound far enough to reach most of the united states']

In [0]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa_vCPU-sklearn.joblib')
cdqa_pipeline.fit_retriever(X=df)

QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir='models', predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1))

In [0]:
query = 'What is the thaad system?'
prediction = cdqa_pipeline.predict(X=query)

3it [00:00, 601.88it/s]


In [0]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: What is the thaad system?
answer: the nature of portraiture itself
title: The Mysterious Metamorphosis of Chuck Close - The New York Times
paragraph: i could not help wondering whether it wa right or fair or even possible to convey in word the man that close wa becoming and i also wondered in a more practical sense how to write about a person in the midst of transmogrifying flux what of any certainty could be noted what insight might be made? it occurs to me now that what this come down to is the nature of portraiture itself

